In [ ]:
import os, sys, re
from glob import glob

def get_info(trial_dir):
    name  = os.path.basename(trial_dir)
    if not os.path.isdir(trial_dir):
        return dict(name=name, status='not found')
    in_trial_dir   = lambda x: os.path.join(trial_dir,x)
    has_tank       = os.path.exists(in_trial_dir('tank.pik'))
    has_background = os.path.exists(in_trial_dir('background.npz'))
    has_trial      = os.path.exists(in_trial_dir('trial.pik'))
    try:
        with open(in_trial_dir('settings.txt')) as f:
            settings = [ [x.strip() for x in line.split('=')] for line in f.readlines() ]
    except:
        settings = None
    try:
        with open(in_trial_dir('log.txt')) as f:
            log = [ line.strip() for line in f.readlines() ]
        if 'Failed' in log:
            status = 'failed'
        elif log[-1] == 'Done':
            status = 'complete'
        else:
            progress = [ line.strip() for line in log if 'Tracking' in line ]
            if len(progress)==0:
                status = 'tracking (0%)'
            else:
                percent = re.findall('[\d.]*%',progress[-1])[0]
                status  = f'tracking ({percent})'
    except:
        status = 'no log'
    
    return locals()

statuses = ['no log','tracking','complete','failed']

In [ ]:
trial_dirs = sorted(glob('../tracking/full_21-01-22/*'))
len(trial_dirs)

In [ ]:
for trial_dir in trial_dirs:
    try:
        globals().update(get_info(trial_dir))
        if status in ['complete','no log']:
#         if 'tracking' not in status:
            continue
        print(f'{status:20s} {name}')
    except:
        print(f'No info for {trial_dir}')

In [ ]:
status_count = { s:0 for s in statuses }

for path in trial_dirs:
    for s in statuses:
        if s in get_info(path)['status']:
            status_count[s] += 1
    
display(status_count)

In [ ]:
# for trial_dir in trial_dirs:
#     locals().update(get_info(trial_dir))
#     if status=='no log':
#         trial_file = os.path.join(trial_dir,'trial.pik')
#         log_file = os.path.join(trial_dir,'log.txt')
#         if os.path.exists(trial_file):
#             print(trial_dir)
#             display(os.listdir(trial_dir))
# #             os.remove(trial_file)
